Using TensorFlow backend.
/home/andres/thesis_project/keras_intel/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [4]:
import gym

In [5]:
env = gym.make('Cart-v4')

In [1]:
import gym
env = gym.make('SpaceInvaders-v4')

In [12]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [3]:
from keras.models import load_model
model = load_model('/media/andres/Baymax/predictor.h5')
encoder = load_model('/media/andres/Baymax/encoder100.h5')

Using TensorFlow backend.
/home/andres/thesis_project/keras2/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
env = gym.make('SpaceInvaders-v4')
state_size = env.observation_space.shape[0] #377
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

NameError: name 'DQNAgent' is not defined

In [13]:
state = env.reset()

In [1]:
import matplotlib.pyplot as plt 
import matplotlib.cm as cm

from skimage import io, color
from skimage.transform import rescale
from skimage.draw import rectangle
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu
from skimage.morphology import closing, square
import h5py

def load_transform(image_path):
    img = io.imread(image_path)
    img = img[25:195,0:160]
    img_gray = color.rgb2gray(img)
    
    thresh = threshold_otsu(img_gray)
    bw = closing(img_gray > thresh, square(3))
    cleared = bw
    label_image = label(cleared

    image_rescaled = rescale(cleared, 1.0 / 3.0, anti_aliasing=False)
    #print(image_rescaled.shape)
    image_reshaped = image_rescaled.reshape(3021)
    image_reshaped[image_reshaped!=0] = 1
    return image_reshaped

In [33]:
import numpy as np

state_transformed = load_transform(state)
encoded = encoder.predict(np.expand_dims(state_transformed, axis=0))
prediction = model.predict(encoded)

/home/andres/thesis/keras/lib/python3.6/site-packages/skimage/transform/_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '
/home/andres/thesis/keras/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [36]:
prediction.argmax()

1

In [28]:
state_transformed.shape

(3021,)

In [1]:
actions = ['NOOP', 'FIRE', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'UPRIGHT', 'UPLEFT', 'DOWNRIGHT', 'DOWNLEFT', 'UPFIRE', 'RIGHTFIRE', 'LEFTFIRE', 'DOWNFIRE', 'UPRIGHTFIRE', 'UPLEFTFIRE', 'DOWNRIGHTFIRE', 'DOWNLEFTFIRE']
env_actions  = ['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

import matplotlib.pyplot as plt
import os 

from skimage import io, color
from skimage.transform import rescale, resize
import numpy as np

from skimage.draw import rectangle
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu
from skimage.morphology import closing, square
from keras.models import load_model
import h5py

model = load_model('/home/andres/Desktop/classifier_space_invaders.h5')

def load_transform(img):
    img = img[25:195,10:150]
    img = resize(img, (60, 52), anti_aliasing=False)
    img = color.rgb2gray(img)
    img[img>=np.mean(img)] = 1
    img[img<np.mean(img)] = 0
    if np.mean(img) > 0.5:
        img = 1-img
    img = img.reshape(1, 60, 52, 1)
    return img

def get_action(state):
    prediction = model.predict(load_transform(state))[0]
    prediction_index = prediction.argmax()
    try: 
        return env_actions.index(actions[prediction_index])
    except:
        return 0
    

Using TensorFlow backend.


In [2]:
env_actions.index('FIRE')

1

In [1]:
import itertools
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import os 

from skimage import io, color
from skimage.transform import rescale, resize
from skimage.draw import rectangle
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu
from skimage.morphology import closing, square
from keras.models import load_model
import h5py

factors = {
    'names': ['guided_exploration', 'reward_goal'],
    'guided_exploration': [0.0],
    'reward_goal': [1000]
}

def generate_experiments(factors, repetitions):
    randomized = []
    for factor_name in factors['names']:
        randomized.append(factors[factor_name])
    experiments_permutations = list(itertools.product(*randomized))*repetitions
    return random.sample(experiments_permutations, len(experiments_permutations))

actions = ['NOOP', 'FIRE', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'UPRIGHT', 'UPLEFT', 'DOWNRIGHT', 'DOWNLEFT', 'UPFIRE', 'RIGHTFIRE', 'LEFTFIRE', 'DOWNFIRE', 'UPRIGHTFIRE', 'UPLEFTFIRE', 'DOWNRIGHTFIRE', 'DOWNLEFTFIRE']
env_actions  = ['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

model = load_model('/home/andres/Desktop/classifier_space_invaders.h5')

def load_transform(img):
    img = img[25:195,10:150]
    img = resize(img, (60, 52), anti_aliasing=False)
    img = color.rgb2gray(img)
    img[img>=np.mean(img)] = 1
    img[img<np.mean(img)] = 0
    if np.mean(img) > 0.5:
        img = 1-img
    img = img.reshape(1, 60, 52, 1)
    return img

def get_action(state):
    prediction = model.predict(state)[0]
    prediction_index = prediction.argmax()
    try: 
        return env_actions.index(actions[prediction_index])
    except:
        return 0

class DQNAgent:
    def __init__(self, state_size, action_size, guided_exploration_rate):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=1024*8)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0 # exploration rate
        self.guided_exploration_rate = 0.0
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.995
        self.learning_rate = 0.05
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Conv2D(16, (8, 8), activation='relu', padding='same', input_shape=(60, 52, 1))) #nb_filter, nb_row, nb_col
        model.add(MaxPooling2D((2, 2), padding='same'))
        model.add(Conv2D(8, (8, 8), activation='relu', padding='same'))
        model.add(MaxPooling2D((2, 2), padding='same'))
        model.add(Conv2D(4, (8, 8), activation='relu', padding='same'))
        model.add(MaxPooling2D((2, 2), padding='same'))
        model.add(Flatten())
        model.add(Dense(self.action_size, activation='softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            if np.random.rand() <= self.guided_exploration_rate:
                return get_action(state)
            else:
                return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, batch_size=64, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


experiments = generate_experiments(factors, 1)

# set constants (does not change between experiments):
env = gym.make('SpaceInvaders-v4')
state_size = 3120
action_size = env.action_space.n
batch_size = 256

for experiment in experiments:
    agent = DQNAgent(state_size, action_size, experiment[0])
    goal = experiment[1]
    goal_completed = False
    num_frames = 0

    while not goal_completed:
        state = env.reset()
        actual_reward = 0
        state = load_transform(state)
        while True:
            num_frames = num_frames + 1
            if np.random.rand() < 0.25:
                action = 0
                missed_frame = True
            else:
                action = agent.act(state)
                missed_frame = False
            next_state, reward, done, _ = env.step(action)
            actual_reward += reward
            reward = reward if not done else -10
            next_state = load_transform(next_state)
            if not missed_frame:
                agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                break
            if actual_reward >= goal:
                goal_completed = True
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
    print("Guided Exploration Rate: {}, Goal: {}, Frames: {}".format(experiment[0], experiment[1], num_frames))
print("Done")

Using TensorFlow backend.
/home/andres/thesis_project/keras_intel/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Guided Exploration Rate: 0.0, Goal: 10, Frames: 107
Guided Exploration Rate: 0.0, Goal: 10, Frames: 104
Guided Exploration Rate: 0.0, Goal: 10, Frames: 155
Guided Exploration Rate: 0.0, Goal: 10, Frames: 168
Guided Exploration Rate: 0.0, Goal: 10, Frames: 66
Done


In [3]:
experiments = generate_experiments(factors, 5)

In [ ]:
# set constants (does not change between experiments):
env = gym.make('SpaceInvaders-v4')
state_size = 3120
action_size = env.action_space.n
batch_size = 256

for experiment in experiments:
    agent = DQNAgent(state_size, action_size, experiment[0])
    goal = experiment[1]
    goal_completed = False
    num_frames = 0

    while not goal_completed:
        state = env.reset()
        actual_reward = 0
        state = load_transform(state)
        while True:
            num_frames = num_frames + 1
            if np.random.rand() < 0.25:
                action = 0
                missed_frame = True
            else:
                action = agent.act(state)
                missed_frame = False
            next_state, reward, done, _ = env.step(action)
            actual_reward += reward
            reward = reward if not done else -10
            next_state = load_transform(next_state)
            if not missed_frame:
                agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                break
            if actual_reward >= goal:
                goal_completed = True
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
    print("Guided Exploration Rate: {}, Goal: {}, Frames: {}".format(experiment[0], experiment[1], num_frames))
print("Done")

/home/andres/thesis_project/keras_intel/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Guided Exploration Rate: 0.75, Goal: 10, Frames: 153


In [12]:
experiments[0]

(0.0, 1000)

In [16]:
model = load_model('/home/andres/Desktop/classifier_space_invaders.h5')

In [13]:
f = h5py.File('/home/andres/Desktop/classifier_space_invaders.h5','r')

In [14]:
f.close()

In [15]:
f

<Closed HDF5 file>

In [17]:
model.to_json()

'{"class_name": "Sequential", "config": {"name": "sequential_11", "layers": [{"class_name": "Conv2D", "config": {"name": "conv2d_1", "trainable": true, "batch_input_shape": [null, 60, 52, 1], "dtype": "float32", "filters": 16, "kernel_size": [8, 8], "strides": [1, 1], "padding": "same", "data_format": "channels_last", "dilation_rate": [1, 1], "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "MaxPooling2D", "config": {"name": "max_pooling2d_1", "trainable": true, "pool_size": [2, 2], "padding": "same", "strides": [2, 2], "data_format": "channels_last"}}, {"class_name": "Conv2D", "config": {"name": "conv2d_2", "trainable": true, "filters": 8, "kern

In [20]:
model.save_weights('/home/andres/Desktop/weights.h5')